In [1]:
import pandas as pd
import warnings
from unidecode import unidecode
from sportsreference.ncaab.roster import Player
from re import search
from tqdm import tqdm, tqdm_notebook
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
from sportsreference.ncaab.roster import Player

In [3]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [4]:
df = pd.read_csv("NBA_data.csv")

In [5]:
#remove accents 
clean_player = [unidecode(player) for player in df["Player"]]
clean_player[:10]

['Kenyon Martin',
 'A.J. Guyton',
 'Jake Voskuhl',
 'Khalid El-Amin',
 'Mike Smith',
 'Eddie House',
 'Eduardo Najera',
 'Lavor Postell',
 'Hanno Mottola',
 'Chris Carrawell']

In [6]:
#make player ids
suffix = "-1"
makes_ids = [player.replace(".", "").replace("'", "").replace(" ", "-").lower()+suffix for player in clean_player]
makes_ids[:10]

['kenyon-martin-1',
 'aj-guyton-1',
 'jake-voskuhl-1',
 'khalid-el-amin-1',
 'mike-smith-1',
 'eddie-house-1',
 'eduardo-najera-1',
 'lavor-postell-1',
 'hanno-mottola-1',
 'chris-carrawell-1']

In [ ]:
#see which players are generating errors when trying to call college data 
for player in makes_ids:
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {makes_ids.index(player)}')
        


#exceptions
deeandre hullett, cc 23
kedrick brown, cc, 78
ronald murray, cc, 93
qyntel woods, cc, 128
jerome beasely, cc, 139
donta smith, cc, 172
luc-richard-mbah-a-moute-1, 337
bill-walker-1, 343
patrick-mills-1, 398
jeff-pendergraph-1, 403
bj-mullens-1, 424
bernard james, cc, 533
thomas-robinson-2, 555
ricky ledo, cc, 579
roy-devyn-marble-1, 644
kyle-anderson-3, 650
stephen-zimmermanjr-1, 720
kahlil-felder-1, 732
jakob-poeltl-1, 748
taurean-prince-1, 750
edrice-adebayo-1, 799
malik-milton-1, 830
raymond-spalding-1, 831
marvin-bagleyiii-1, 839
lonnie-walker-2, 853

In [7]:
#make list to fix incorrect player id's 
replace = [('luc-richard-mbah-a-moute-1', 337), ('bill-walker-1', 343), ('patrick-mills-1', 398), ('jeff-pendergraph-1', 403), 
          ('bj-mullens-1', 424),('thomas-robinson-2', 555), ('roy-devyn-marble-1', 644), ('kyle-anderson-3', 650), 
           ('stephen-zimmermanjr-1', 720), ('kahlil-felder-1', 732),('jakob-poeltl-1', 748), ('taurean-prince-1', 750),
           ('edrice-adebayo-1', 799), ('malik-milton-1', 830),('raymond-spalding-1', 831), ('marvin-bagleyiii-1', 839),
           ('lonnie-walker-2', 853)]

In [8]:
#replace incorrect id's
for player in replace:
    makes_ids[player[1]] = player[0]

In [9]:
#test corrected list 
makes_ids[337]

'luc-richard-mbah-a-moute-1'

In [10]:
#list players to remove
remove = ['deeandre-hulett-1', 'kedrick-brown-1', 'ronald-murray-1', 'qyntel-woods-1',
'jerome-beasley-1', 'donta-smith-1','bernard-james-1', 'ricky-ledo-1']
no_cc = [player for player in makes_ids if player not in remove] 
len(no_cc)

855

In [11]:
#list index of players to remove
remove_nums = [23, 78, 93, 128, 139, 172, 533, 579]

In [12]:
#remove same index rows from df
clean_df = df.drop(remove_nums)

In [13]:
#add Player_id column to lean NBA stats df
clean_df['Player_id'] = no_cc

In [14]:
#get colleges of players to make sure we have the correct players
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [15]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [16]:
#add to clean_df
clean_df['College_id'] = college_list

In [17]:
#fix college abbreviations
clean_df["College_id"]= (clean_df["College_id"]
.replace("connecticut", "uconn").replace("louisiana state", "lsu").replace("southern california", "usc")
.replace('north carolina', 'unc').replace('southern methodist', 'smu').replace('brigham young', 'byu')
.replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
.replace('massachusetts', 'umass').replace('pittsburgh', 'pitt').replace('louisiana lafayette', 'la-lafayette'))


In [18]:
#convert College column to lower case 
clean_df['College'] = clean_df['College'].str.lower() 

In [19]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['College_id']]
clean_df['word'] = word

In [20]:
wrong_players = []
for index, row in clean_df.iterrows():
    if search(row["word"], row["College"]):
        print(row["Player"], "at ", index, " matches")
    else:
        print(row["Player"], "at ", index, " does NOT match")
        wrong_players.append((row["Player"]))


Kenyon Martin at  0  matches
A.J. Guyton at  1  matches
Jake Voskuhl at  2  matches
Khalid El-Amin at  3  matches
Mike Smith at  4  matches
Eddie House at  5  matches
Eduardo Nájera at  6  matches
Lavor Postell at  7  matches
Hanno Möttölä at  8  matches
Chris Carrawell at  9  matches
Michael Redd at  10  matches
Dan Langhi at  11  matches
Jabari Smith at  12  matches
Mark Karcher at  13  matches
Jason Hart at  14  matches
Kaniel Dickens at  15  matches
Ernest Brown at  16  does NOT match
Dan McClintock at  17  matches
Cory Hightower at  18  does NOT match
Chris Porter at  19  matches
Jaquay Walls at  20  matches
Scoonie Penn at  21  matches
Pete Mickeal at  22  matches
Brian Cardinal at  24  matches
Erick Barkley at  25  matches
Mark Madsen at  26  matches
Stromile Swift at  27  matches
Marcus Fizer at  28  matches
Mike Miller at  29  matches
DerMarr Johnson at  30  matches
Chris Mihm at  31  matches
Jamal Crawford at  32  matches
Keyon Dooling at  33  matches
Jérôme Moïso at  34  mat

Maurice Ager at  275  matches
Jordan Farmar at  276  matches
Kyle Lowry at  277  matches
Josh Boone at  278  matches
Shannon Brown at  279  matches
Rajon Rondo at  280  matches
Renaldo Balkman at  281  matches
Quincy Douby at  282  matches
Shawne Williams at  283  matches
Marcus Williams at  284  matches
Adam Haluska at  285  matches
Derrick Byars at  286  matches
Chris Richard at  287  matches
Marcus Williams at  288  does NOT match
Josh McRoberts at  289  matches
Jermareo Davidson at  290  matches
Glen Davis at  291  matches
Nick Fazekas at  292  matches
Reyshawn Terry at  293  matches
Jared Jordan at  294  matches
Taurean Green at  295  matches
Dominic McGuire at  296  matches
Aaron Gray at  297  matches
JamesOn Curry at  298  matches
Demetris Nichols at  299  matches
Herbert Hill at  300  matches
Ramon Sessions at  301  matches
Sammy Mejia at  302  matches
D.J. Strawberry at  303  matches
Gabe Pruitt at  304  matches
Stéphane Lasme at  305  matches
Carl Landry at  306  matches
Alan

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  matches
Isaiah Thomas at  503  matches
Cory Joseph at  504  matches
Kyrie Irving at  505  matches
Derrick Williams at  506  does NOT match
Tristan Thompson at  507  does NOT match
Brandon Knight at  508  matches
Kemba Walker at  509  matches
Jimmer Fredette at  510  matches
Klay Thompson at  511  matches
Alec Burks at  512  matches
Markieff Morris at  513  matches
Marcus Morris at  514  matches
Jimmy Butler at  515  matches
Nikola Vučević at  516  matches
Kawhi Leonard at  517  matches
Norris Cole at  518  matches
JaJuan Johnson at  519  matches
Jordan Hamilton at  520  does NOT match
Reggie Jackson at  521  matches
MarShon Brooks at  522  matches
Nolan Smith at  523  matches
Tobias Harris at  524  matches
Chris Singleton at  525  matches
Iman Shumpert at  526  matches
Kenneth Faried at  527  matches
Will Barton at  528  matches
Mike Scott at  529  does NOT match
Doron Lamb at  530  matches
Tyshawn Taylor at  531  matches
Khris Middleton at  532  matches
Quincy Acy at  534  matches
O

Tyler Dorsey at  762  matches
Damyean Dotson at  763  does NOT match
Thomas Bryant at  764  matches
Dwayne Bacon at  765  matches
Wesley Iwundu at  766  matches
Jordan Bell at  767  matches
Semi Ojeleye at  768  matches
Ivan Rabb at  769  matches
Frank Mason at  770  matches
Jawun Evans at  771  matches
Sterling Brown at  772  does NOT match
Sindarius Thornwell at  773  matches
Monte Morris at  774  matches
Edmond Sumner at  775  matches
Kadeem Allen at  776  matches
Alec Peters at  777  matches
Nigel Williams-Goss at  778  matches
Jabari Bird at  779  matches
Jaron Blossomgame at  780  matches
Davon Reed at  781  matches
Ike Anigbogu at  782  matches
Frank Jackson at  783  does NOT match
Dillon Brooks at  784  matches
Derrick White at  785  matches
Markelle Fultz at  786  matches
Lonzo Ball at  787  matches
Jayson Tatum at  788  matches
Josh Jackson at  789  does NOT match
De'Aaron Fox at  790  matches
Jonathan Isaac at  791  matches
Lauri Markkanen at  792  matches
Josh Hart at  793 

In [21]:
len(wrong_players)

54

In [22]:
select = clean_df["Player"].isin(wrong_players)
wrong_df = clean_df[select]

In [23]:
wrong_df.head(54)

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class,Player_id,College_id,word
16,-0.2,7.0,Ernest Brown,3,2000,52,1.0,-0.349,-24.7,-0.1,MIA,indian hills community college,1,21,3,Bust,ernest-brown-1,alabama,alabama
18,0.0,0.0,Cory Hightower,0,2000,54,0.0,0.000,0.0,0.0,SAS,indian hills community college,0,0,0,Bust,cory-hightower-1,presbyterian,presbyterian
61,0.0,9.8,Ken Johnson,16,2001,49,2.0,-0.002,-10.0,-0.3,MIA,ohio state,1,156,32,Bust,ken-johnson-1,michigan state,michigan
115,58.5,27.7,Mike Dunleavy,986,2002,3,11.2,0.103,0.3,15.9,GSW,duke,15,27340,11048,Above Avg,mike-dunleavy-1,south carolina,south
167,0.0,0.0,David Young,0,2004,41,0.0,0.000,0.0,0.0,SEA,nc central,0,0,0,Bust,david-young-1,clemson,clemson
211,0.0,9.2,Robert Whaley,23,2005,51,2.1,0.004,-3.5,-0.1,UTA,walsh university,1,212,49,Bust,robert-whaley-1,cincinnati,cincinnati
246,1.1,9.5,Bobby Jones,91,2006,37,3.2,0.060,-3.5,-0.3,MIN,washington,2,867,288,Bust,bobby-jones-1,unc,unc
258,0.1,10.5,Dee Brown,68,2006,46,2.1,0.008,-4.1,-0.4,UTA,illinois,2,711,140,Bust,dee-brown-1,jacksonville,jacksonville
284,0.0,15.2,Marcus Williams,203,2006,22,5.6,-0.001,-4.6,-2.0,NJN,uconn,4,3095,1128,Below Avg,marcus-williams-1,uconn,uconn
288,-0.1,3.0,Marcus Williams,13,2007,33,1.1,-0.084,-7.5,-0.1,SAS,arizona,2,39,14,Bust,marcus-williams-1,uconn,uconn


In [24]:
#create list of players to remove
remove2 = ['ernest-brown-1', 'cory-hightower-1', ]

In [25]:
#create list of player ids to update
replace = [('ken-johnson-1', 'ken-johnson-2'), ('mike-dunleavy-1', 'mike-dunleavy-2'), ('david-young-1','dave-young-1'),
          ('bobby-jones-1', 'bobby-jones-2'), ('dee-brown-1', 'dee-brown-2'), ('gerald-henderson-1', 'gerald-henderson-2'),
          ('james-johnson-1', 'james-johnson-2'), ('wesley-johnson-1', 'wes-johnson-1'),
          ('james-anderson-1', 'james-anderson-3'), ('derrick-williams-1', 'derrick-williams-2'),
          ('tristan-thompson-1','tristan-thompson-2'), ('jordan-hamilton-1', 'jordan-hamilton-2'),
          ('mike-scott-1', 'mike-scott-4'), ('justin-hamilton-1', 'justin-hamilton-2'), ('anthony-davis-1', 'anthony-davis-5'),
          ('jared-cunningham-1', 'jared-cunningham-2'), ('ray-mccallum-1', 'ray-mccallum-2'), ('glen-rice-1', 'glen-rice-jr-1'),
          ('ryan-kelly-1', 'ryan-kelly-3'), ('reggie-bullock-1', 'reggie-bullock-2'), ('tim-hardaway-1', 'tim-hardaway-jr-1'),
          ('glenn-robinson-1', 'glenn-robinson-2'),('joe-harris-1', 'joe-harris--1'),
          ('marcus-thornton-1', 'marcus-thornton-3'), ('joe-young-1', 'joseph-young-1'),
          ('josh-richardson-1', 'josh-richardson-2'), ('anthony-brown-1', 'anthony-brown-5'), ('aaron-white-1','aaron-white-3'),
          ('devin-booker-1', 'devin-booker-2'), ('larry-nance-1', 'larry-nance-2'),
          ('demetrius-jackson-1', 'demetrius-jackson-2'), ('sterling-brown-1', 'sterling-brown-2'),
          ('frank-jackson-1', 'frank-jackson-2'), ('josh-jackson-1', 'josh-jackson-2'), ('dennis-smith-1', 'dennis-smithjr-1'),
          ('justin-jackson-1', 'justin-jackson-4'), ('justin-jackson-1', 'justin-jackson-5'),
          ('bruce-brown-1', 'bruce-brown-2'), ('gary-trent-1', 'gary-trentjr-1'), ('vince-edwards-1', 'vince-edwards-2'),
          ('troy-brown-1', 'troy-brown-5'), ('jaren-jackson-1', 'jaren-jacksonjr-1'), 
          ('wendell-carter-1', 'wendell-carterjr-1'),('jerome-robinson-1', 'jerome-robinson-2'),
          ('marcus-williams-1', 'marcus-williams-2'), ('michael-porter-1', 'michael-porterjr-1'),
          ('robert-williams-1', 'robert-williams-3')
          ]

In [26]:
#update player ids in clean_df
for player in replace:
    clean_df.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [27]:
#input justin jackson 5
clean_df.loc[812, 'Player_id'] = 'justin-jackson-5'

In [28]:
#remove players not in NCAA
clean_df = clean_df[~clean_df['Player_id'].isin(remove2)]

In [29]:
#test player ids one more time
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [30]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [31]:
#add to clean_df
clean_df['College_id'] = college_list

In [32]:
#fix college abbreviations
clean_df["College_id"]= (clean_df["College_id"]
.replace("connecticut", "uconn").replace("louisiana state", "lsu").replace("southern california", "usc")
.replace('north carolina', 'unc').replace('southern methodist', 'smu').replace('brigham young', 'byu')
.replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
.replace('massachusetts', 'umass').replace('pittsburgh', 'pitt').replace('louisiana lafayette', 'la-lafayette'))

In [33]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['College_id']]
clean_df['word'] = word

In [34]:
#test player ids one more time
wrong_players = []
for index, row in clean_df.iterrows():
    if search(row["word"], row["College"]):
        print(row["Player"], "at ", index, " matches")
    else:
        print(row["Player"], "at ", index, " does NOT match")
        wrong_players.append((row["Player"], index))

Kenyon Martin at  0  matches
A.J. Guyton at  1  matches
Jake Voskuhl at  2  matches
Khalid El-Amin at  3  matches
Mike Smith at  4  matches
Eddie House at  5  matches
Eduardo Nájera at  6  matches
Lavor Postell at  7  matches
Hanno Möttölä at  8  matches
Chris Carrawell at  9  matches
Michael Redd at  10  matches
Dan Langhi at  11  matches
Jabari Smith at  12  matches
Mark Karcher at  13  matches
Jason Hart at  14  matches
Kaniel Dickens at  15  matches
Dan McClintock at  17  matches
Chris Porter at  19  matches
Jaquay Walls at  20  matches
Scoonie Penn at  21  matches
Pete Mickeal at  22  matches
Brian Cardinal at  24  matches
Erick Barkley at  25  matches
Mark Madsen at  26  matches
Stromile Swift at  27  matches
Marcus Fizer at  28  matches
Mike Miller at  29  matches
DerMarr Johnson at  30  matches
Chris Mihm at  31  matches
Jamal Crawford at  32  matches
Keyon Dooling at  33  matches
Jérôme Moïso at  34  matches
Etan Thomas at  35  matches
Courtney Alexander at  36  matches
Mateen

Shannon Brown at  279  matches
Rajon Rondo at  280  matches
Renaldo Balkman at  281  matches
Quincy Douby at  282  matches
Shawne Williams at  283  matches
Marcus Williams at  284  does NOT match
Adam Haluska at  285  matches
Derrick Byars at  286  matches
Chris Richard at  287  matches
Marcus Williams at  288  matches
Josh McRoberts at  289  matches
Jermareo Davidson at  290  matches
Glen Davis at  291  matches
Nick Fazekas at  292  matches
Reyshawn Terry at  293  matches
Jared Jordan at  294  matches
Taurean Green at  295  matches
Dominic McGuire at  296  matches
Aaron Gray at  297  matches
JamesOn Curry at  298  matches
Demetris Nichols at  299  matches
Herbert Hill at  300  matches
Ramon Sessions at  301  matches
Sammy Mejia at  302  matches
D.J. Strawberry at  303  matches
Gabe Pruitt at  304  matches
Stéphane Lasme at  305  matches
Carl Landry at  306  matches
Alando Tucker at  307  matches
Greg Oden at  308  matches
Kevin Durant at  309  matches
Al Horford at  310  matches
Mike 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Alec Burks at  512  matches
Markieff Morris at  513  matches
Marcus Morris at  514  matches
Jimmy Butler at  515  matches
Nikola Vučević at  516  matches
Kawhi Leonard at  517  matches
Norris Cole at  518  matches
JaJuan Johnson at  519  matches
Jordan Hamilton at  520  matches
Reggie Jackson at  521  matches
MarShon Brooks at  522  matches
Nolan Smith at  523  matches
Tobias Harris at  524  matches
Chris Singleton at  525  matches
Iman Shumpert at  526  matches
Kenneth Faried at  527  matches
Will Barton at  528  matches
Mike Scott at  529  matches
Doron Lamb at  530  matches
Tyshawn Taylor at  531  matches
Khris Middleton at  532  matches
Quincy Acy at  534  matches
Orlando Johnson at  535  does NOT match
Draymond Green at  536  matches
Jae Crowder at  537  matches
Kim English at  538  matches
Quincy Miller at  539  matches
Justin Hamilton at  540  matches
Kevin Murphy at  541  matches
Kyle O'Quinn at  542  matches
Kris Joseph at  543  matches
Darius Johnson-Odom at  544  matches
Rob

Kadeem Allen at  776  matches
Alec Peters at  777  matches
Nigel Williams-Goss at  778  matches
Jabari Bird at  779  matches
Jaron Blossomgame at  780  matches
Davon Reed at  781  matches
Ike Anigbogu at  782  matches
Frank Jackson at  783  matches
Dillon Brooks at  784  matches
Derrick White at  785  matches
Markelle Fultz at  786  matches
Lonzo Ball at  787  matches
Jayson Tatum at  788  matches
Josh Jackson at  789  matches
De'Aaron Fox at  790  matches
Jonathan Isaac at  791  matches
Lauri Markkanen at  792  matches
Josh Hart at  793  matches
Dennis Smith at  794  matches
Zach Collins at  795  matches
Malik Monk at  796  matches
Luke Kennard at  797  matches
Donovan Mitchell at  798  matches
Bam Adebayo at  799  matches
Justin Patton at  800  matches
Justin Jackson at  801  matches
Tony Bradley at  802  matches
Kyle Kuzma at  803  matches
Caleb Swanigan at  804  matches
Tyler Lydon at  805  matches
OG Anunoby at  806  matches
Harry Giles at  807  matches
John Collins at  808  match

In [35]:
len(wrong_players)

8

In [36]:
#all 'wrong players' are fine in this case.
wrong_players

[('David Young', 167),
 ('Robert Whaley', 211),
 ('Marcus Williams', 284),
 ('Trent Plaisted', 354),
 ('Marcus Thornton', 382),
 ('Orlando Johnson', 535),
 ('Damyean Dotson', 763),
 ('Kevin Hervey', 832)]

In [ ]:
# d = []
# for p in game.players.passing():
#     d.append({'Player': p, 'Team': p.team, 'Passer Rating':
#         p.passer_rating()})

# pd.DataFrame(d)

In [62]:
#create  list to hold dictionary data and list for desired ncaa stats
ncaa_data = []

stats = ['player_id','name','assist_percentage', 'assists', 'block_percentage','blocks','effective_field_goal_percentage',
        'field_goal_percentage', 'free_throw_attempt_rate', 'free_throw_percentage', 'minutes_played',
        'points', 'steal_percentage', 'steals', 'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
         'total_rebounds', 'true_shooting_percentage', 'turnover_percentage','turnovers', 'usage_percentage', 'box_plus_minus',
        'conference', 'games_played', 'height', 'win_shares_per_40_minutes', 'weight']



In [64]:
len(stats)

28

In [63]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(clean_df["Player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)
    

weight nan
weight nan
weight nan
weight nan
weight nan



In [65]:
#convert ncaa_data to df
ncaa_df = pd.DataFrame(ncaa_data)


In [68]:
ncaa_df

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,conference,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,name,player_id,points,steal_percentage,steals,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,usage_percentage,weight,win_shares_per_40_minutes,college,class
0,NaN,142,NaN,292,NaN,,0.587,0.586,0.494,0.581,116,6-9,2042.0,Kenyon Martin,kenyon-martin-1,1279,NaN,124,0.222,2.0,NaN,869,0.592,14.3,139.0,NaN,234.0,0.346,cincinnati,Above Avg
1,NaN,403,NaN,52,NaN,,0.541,0.455,0.249,0.790,128,6-1,3260.0,A.J. Guyton,aj-guyton-1,2100,NaN,129,0.414,283.0,NaN,429,0.572,13.6,218.0,NaN,180.0,0.145,indiana,Below Avg
2,NaN,124,NaN,193,NaN,,0.542,0.542,0.555,0.656,138,6-11,2184.0,Jake Voskuhl,jake-voskuhl-1,863,NaN,86,NaN,0.0,NaN,880,0.573,23.0,160.0,NaN,245.0,0.237,uconn,Below Avg
3,NaN,479,NaN,10,NaN,,0.486,0.416,0.257,0.822,108,5-10,2149.0,Khalid El-Amin,khalid-el-amin-1,1650,NaN,186,0.355,195.0,NaN,319,0.527,15.6,188.0,NaN,200.0,0.158,uconn,Bust
4,NaN,147,NaN,42,NaN,,0.507,0.428,0.357,0.769,55,6-8,1938.0,Mike Smith,mike-smith-1,1062,NaN,90,0.318,130.0,NaN,449,0.551,20.0,241.0,NaN,195.0,0.165,louisiana monroe,Bust
5,NaN,405,NaN,19,NaN,,0.495,0.425,0.187,0.798,124,6-1,4136.0,Eddie House,eddie-house-1,2044,NaN,258,0.366,250.0,NaN,502,0.523,10.8,195.0,NaN,180.0,0.113,arizona state,Above Avg
6,NaN,215,NaN,89,NaN,,0.460,0.430,0.354,0.669,126,6-8,3001.0,Eduardo Najera,eduardo-najera-1,1646,NaN,193,0.288,85.0,NaN,910,0.495,13.1,201.0,NaN,240.0,0.183,oklahoma,Above Avg
7,NaN,205,NaN,58,NaN,,0.470,0.430,0.383,0.789,129,6-5,2845.0,Lavor Postell,lavor-postell-1,1427,NaN,129,0.326,92.0,NaN,716,0.526,13.8,172.0,NaN,215.0,0.115,st johns ny,Below Avg
8,NaN,139,NaN,50,NaN,,0.533,0.500,0.415,0.774,120,6-11,2162.0,Hanno Mottola,hanno-mottola-1,1492,NaN,37,0.333,71.0,NaN,551,0.580,14.9,159.0,NaN,247.0,0.207,utah,Bust
9,NaN,309,NaN,114,NaN,,0.514,0.486,0.415,0.667,136,6-6,NaN,Chris Carrawell,chris-carrawell-1,1455,NaN,105,0.360,62.0,NaN,608,0.545,NaN,NaN,NaN,210.0,NaN,duke,Bust


In [67]:
#add college to ncaa_df
ncaa_df["college"] = clean_df["College_id"]
ncaa_df["class"] = clean_df["Class"]

In [69]:
#create new dataframe which re-orders columns and drops those with a lot of NaN's
ncaa2_df = ncaa_df[["player_id", "name",  "class", "college", "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points","steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "weight", "win_shares_per_40_minutes" ]]

In [71]:
#stats per 40  list
per_40 = [ "assists", "blocks", "points", "steals", "three_pointers", "total_rebounds", "turnovers"]

In [72]:
#add suffix for column titles
suff = "_per_40"
for_40 = [stat+suff for stat in per_40]

In [75]:
#zip two lists together
combined_40 = list(zip(per_40,for_40))

In [78]:
#check combined list
combined_40

[('assists', 'assists_per_40'),
 ('blocks', 'blocks_per_40'),
 ('points', 'points_per_40'),
 ('steals', 'steals_per_40'),
 ('three_pointers', 'three_pointers_per_40'),
 ('total_rebounds', 'total_rebounds_per_40'),
 ('turnovers', 'turnovers_per_40')]

In [79]:
#run for loop to create per_40 columns
for stat in combined_40:
    ncaa2_df[stat[1]] = (ncaa2_df[stat[0]] /ncaa2_df["minutes_played"])*40

In [80]:
ncaa2_df.head()

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,weight,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Above Avg,cincinnati,142,292,0.587,0.586,0.494,0.581,116,6-9,2042.0,1279,124,0.222,2.0,869,0.592,14.3,139.0,234.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Below Avg,indiana,403,52,0.541,0.455,0.249,0.790,128,6-1,3260.0,2100,129,0.414,283.0,429,0.572,13.6,218.0,180.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Below Avg,uconn,124,193,0.542,0.542,0.555,0.656,138,6-11,2184.0,863,86,NaN,0.0,880,0.573,23.0,160.0,245.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,479,10,0.486,0.416,0.257,0.822,108,5-10,2149.0,1650,186,0.355,195.0,319,0.527,15.6,188.0,200.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,louisiana monroe,147,42,0.507,0.428,0.357,0.769,55,6-8,1938.0,1062,90,0.318,130.0,449,0.551,20.0,241.0,195.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [81]:
ncaa2_df.to_csv("NCAA_data.csv",index = False, header = True)